# Introduction

Prerequisites: CTCN 101

The normalised difference vegeation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing assessed greenness of vegetation.

>$NDVI=(NIR-R)/(NIR+R)$

In [0]:
def addNDVI(img):
    """A function to compute Sentinel-2 NDVI."""
    ndvi = img.expression('float(b("B8") - b("B4")) / (b("B8") + b("B4"))').rename('ndvi')   
    return img.addBands([ndvi])

In [0]:
# Function to mask clouds using the Sentinel-2 SCL band.
def S2CloudMaskSCL(img):
  qa = img.select('SCL')
  mask_band = qa.eq(3).Or(qa.eq(8)).Or(qa.eq(9)).Or(qa.eq(10)).Or(qa.eq(11))
  mask = mask_band.eq(0)
  # Return the masked and scaled data.
  return img.updateMask(mask)
def ESAcloudMask(img):
    ''' Sentinel-2 Bits 10 & 11 are clouds & cirrus, so set to 0. '''
    qa = img.select('QA60')
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return img.updateMask(clear) 

In [0]:
def s2col_monthly(feat):
      bandNamesAll = ['ndvi']
      monthDifference = ee.Date('2019-01-01').advance(1, 'month').millis().subtract(ee.Date('2019-01-01').millis());
      listMap = ee.List.sequence(ee.Date('2019-01-01').millis(), ee.Date('2020-01-01').millis(), monthDifference);

      s2monthIC = (ee.ImageCollection.fromImages(listMap.map(lambda dateMillis: (
         (ee.ImageCollection('COPERNICUS/S2_SR')
                            .filterBounds(feat)
                            .filter(ee.Filter.And(
                              ee.Filter.date(ee.Date(dateMillis), ee.Date(dateMillis).advance(1, 'month')),
                              ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90)))
                            .map(S2CloudMaskSCL)
                            .map(ESAcloudMask)
                            
                            ).median())
      )))
      
      return s2monthIC.map(addNDVI).select(bandNamesAll)

In [0]:
AOI = ee.Geometry.Polygon(
        [[[
              89.40673828125,
              21.37124437061831
            ],
            [
              90.90087890624999,
              21.37124437061831
            ],
            [
              90.90087890624999,
              22.471954507739227
            ],
            [
              89.40673828125,
              22.471954507739227
            ],
            [
              89.40673828125,
              21.37124437061831
            ]]])

In [0]:
# Fetch a Landsat image.
img = s2col_monthly(AOI).toBands().clip(AOI)

In [83]:
monthID = 5

mapid = img.select(monthID).getMapId({'min': -0.8, 'max': 0.8, 'palette': ['fffdcd', 'e1cd73', 'aaac20', '5f920c', '187328', '144b2a', '172313']})
map = folium.Map(location=[22., 89.], zoom_start=7)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [0]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt
from pprint import pprint

# Select NDVI, scale them, and sample 500 points.
water = ee.Geometry.Point([
          89.89700317382812,
          22.11999383809455
        ])
samp_fc = img.sample(region=water,scale=10)

# Save server-side ee.List as a client-side Python list.
samp_data = samp_fc.toDictionary().values().getInfo()

# # Display a scatter plot of Red-NIR sample pairs using matplotlib.
# plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
# plt.xlabel('Red', fontsize=12)
# plt.ylabel('NIR', fontsize=12)
# plt.show()

In [128]:
pprint ({'test:':samp_data[0]})

{'test:': ['0_ndvi',
           '1_ndvi',
           '2_ndvi',
           '3_ndvi',
           '4_ndvi',
           '5_ndvi',
           '6_ndvi',
           '7_ndvi',
           '8_ndvi',
           '9_ndvi',
           '10_ndvi',
           '11_ndvi']}


In [0]:
# Arrange the sample as a list of lists.
samp_fc = img.sample(scale=10, numPixels=500)
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(12), ['0_ndvi','1_ndvi','2_ndvi','3_ndvi','4_ndvi','5_ndvi','6_ndvi','7_ndvi','8_ndvi','9_ndvi','10_ndvi','11_ndvi'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

In [116]:
print (len(samp_data[0]))

270
